In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.layer2 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.output_layer(x))
        return x

def train_and_evaluate_nn(X_train, y_train, X_test, y_test, description):
    model = NeuralNetwork(X_train.shape[1]).to(device)
    
    # Convert numpy arrays to torch tensors
    X_train_torch = torch.tensor(X_train, dtype=torch.float32)
    y_train_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test_torch = torch.tensor(X_test, dtype=torch.float32)
    y_test_torch = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    
    # Create TensorDatasets and DataLoaders with a reduced batch size
    train_dataset = TensorDataset(X_train_torch, y_train_torch)
    test_dataset = TensorDataset(X_test_torch, y_test_torch)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    
    # Training the model
    for epoch in range(10):  # Loop over the dataset multiple times
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_accuracy = 0.0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                val_running_accuracy += (predicted == labels).sum().item()
        
        val_accuracy = val_running_accuracy / total
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Accuracy: {val_accuracy}')

    
    # Evaluating the model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            predictions.extend(predicted.cpu().numpy())
    
    accuracy = accuracy_score(y_test, np.round(predictions))
    f1 = f1_score(y_test, np.round(predictions))
    
    print(f"Results for {description}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}\n")

    # Save the model
    model_dir = "Models/Dense_torch"
    description_formatted = description.replace(' ', '_').lower()
    model_filename = os.path.join(model_dir, f"{description_formatted}_model.pth")
    os.makedirs(model_dir, exist_ok=True)
    torch.save(model.state_dict(), model_filename)
    print(f"Model saved as {model_filename}")

# Example usage
# Ensure your data (X_train, y_train, X_test, y_test) is loaded and preprocessed correctly before calling the function.
# train_and_evaluate_nn(X_train, y_train, X_test, y_test, "example_description")


In [ ]:
data_short = np.load('Data/lorfs_100.npz')

X_train_short = data_short['X_train']
X_test_short = data_short['X_test']
y_train_short = data_short['y_train']
y_test_short = data_short['y_test']
X_train_short = X_train_short.reshape(X_train_short.shape[0], -1)
X_test_short = X_test_short.reshape(X_test_short.shape[0], -1)
train_and_evaluate_nn(X_train_short, y_train_short, X_test_short, y_test_short, "Short Dataset")



In [ ]:
del X_test_short, X_train_short, y_test_short, y_train_short
data_medium = np.load('Data/lorfs_400.npz')

X_train_medium = data_medium['X_train']
X_test_medium = data_medium['X_test']
y_train_medium = data_medium['y_train']
y_test_medium = data_medium['y_test']
X_train_medium = X_train_medium.reshape(X_train_medium.shape[0], -1)
X_test_medium = X_test_medium.reshape(X_test_medium.shape[0], -1)

train_and_evaluate_nn(X_train_medium, y_train_medium, X_test_medium, y_test_medium, "Medium Dataset")


In [ ]:
del X_test_medium, X_train_medium, y_test_medium, y_train_medium
data_long = np.load('Data/lorfs_1000.npz')

X_train_long = data_long['X_train']
X_test_long = data_long['X_test']
y_train_long = data_long['y_train']
y_test_long = data_long['y_test']
X_train_long = X_train_long.reshape(X_train_long.shape[0], -1)
X_test_long = X_test_long.reshape(X_test_long.shape[0], -1)

train_and_evaluate_nn(X_train_long, y_train_long, X_test_long, y_test_long, "Long Dataset")